In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from data_loader import load_track_data, load_race_data, transform_coordinates
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import math
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import LineString
import plotly.graph_objects as go


In [36]:
track_left_side, track_right_side = load_track_data(distance_pr_dot = 1)

track_left_side = track_left_side[:1300]
track_right_side = track_right_side[:1300]

track_left_side = transform_coordinates(track_left_side)
track_right_side = transform_coordinates(track_right_side)

track_left_side.reset_index(drop=True, inplace=True)
track_right_side.reset_index(drop=True, inplace=True)

gdf_left = gpd.GeoDataFrame(track_left_side, geometry=[Point(x, y) for x, y in zip(track_left_side["X-Coords"], track_left_side["Y-Coords"])])
gdf_right = gpd.GeoDataFrame(track_right_side, geometry=[Point(x, y) for x,y in zip(track_right_side["X-Coords"], track_right_side["Y-Coords"])])

gdf_left

c:\GitHub\RacingTrajectories\data_loader.py:5: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.

c:\GitHub\RacingTrajectories\data_loader.py:6: ParserWarning:

Length of header or names does not match length of data. This leads to a loss of data with index_col=False.



,Timestamp,Y-Coords,Z-Coords,X-Coords,geometry
0,12.072,50.335608,63.983009,6.947884,POINT (6.94788 50.33561)
1,14.259,50.335602,63.973942,6.947874,POINT (6.94787 50.33560)
2,14.685,50.335595,63.964184,6.947863,POINT (6.94786 50.33560)
3,0.192,50.335589,63.956116,6.947853,POINT (6.94785 50.33559)
4,0.456,50.335583,63.948250,6.947842,POINT (6.94784 50.33558)
...,...,...,...,...,...
1295,110.391,50.330551,49.595081,6.939633,POINT (6.93963 50.33055)
1296,110.454,50.330542,49.581982,6.939629,POINT (6.93963 50.33054)
1297,110.517,50.330533,49.567482,6.939626,POINT (6.93963 50.33053)
1298,110.583,50.330524,49.557781,6.939623,POINT (6.93962 50.33052)


In [37]:
# Create spatial indices for your GeoDataFrames. That will optimize spatial queries, like finding nearest neighbors:

sindex_left = gdf_left.sindex
sindex_right = gdf_right.sindex


In [64]:

nearest_points = []
spacing = 0.00004 #(1m or 10cm ?)

prev_left_point = gdf_left.geometry[0]
for left_point in gdf_left.geometry:
    # if distance from prev_left_point to left_point is less than  spacing  then skip
    if prev_left_point.distance(left_point) < spacing:
        continue
    else: 
        prev_left_point = left_point

    # Find the index of the nearest point in the right GeoDataFrame
    possible_matches_index = list(sindex_right.nearest(left_point, 0))  # This should return indices of the nearest points

    if possible_matches_index:  # Making sure there is at least one match
        nearest_right_point_index = possible_matches_index[1]  # Take the first index if multiple are returned
        # print("nearest_right_point_index: ", nearest_right_point_index)
        nearest_right_point = list(gdf_right.iloc[nearest_right_point_index].geometry)[0]  # Access the geometry of the nearest point
        # print("nearest_right_point: ", nearest_right_point)
        nearest_points.append((left_point, nearest_right_point))

# prev_right_point = gdf_right.geometry[0]
# for right_point in gdf_right.geometry:
#     # if distance from prev_right_point to right_point is less than  spacing  then skip
#     if prev_right_point.distance(right_point) < spacing:
#         continue
#     # Find the index of the nearest point in the right GeoDataFrame
#     possible_matches_index = list(sindex_left.nearest(right_point, 0))  # This should return indices of the nearest points

#     if possible_matches_index:  # Making sure there is at least one match
#         nearest_left_point_index = possible_matches_index[1]  # Take the first index if multiple are returned
#         # print("nearest_right_point_index: ", nearest_right_point_index)
#         nearest_left_point = list(gdf_left.iloc[nearest_left_point_index].geometry)[0]  # Access the geometry of the nearest point
#         # print("nearest_right_point: ", nearest_right_point)
#         nearest_points.append((nearest_left_point, right_point))


nearest_points

# Assuming nearest_points is a list of tuples containing Shapely Points
nearest_lines_gdf = gpd.GeoDataFrame(geometry=[LineString([p1, p2]) for p1, p2 in nearest_points if p2 is not None])

nearest_lines_gdf


,geometry
0,"LINESTRING (6.94784 50.33558, 6.94776 50.33570)"
1,"LINESTRING (6.94780 50.33556, 6.94772 50.33567)"
2,"LINESTRING (6.94776 50.33553, 6.94769 50.33565)"
3,"LINESTRING (6.94772 50.33550, 6.94765 50.33562)"
4,"LINESTRING (6.94767 50.33548, 6.94760 50.33559)"
...,...
360,"LINESTRING (6.93969 50.33069, 6.93956 50.33073)"
361,"LINESTRING (6.93967 50.33065, 6.93954 50.33068)"
362,"LINESTRING (6.93966 50.33061, 6.93954 50.33065)"
363,"LINESTRING (6.93964 50.33057, 6.93953 50.33064)"


In [65]:
import plotly.graph_objects as go

# Initialize a plotly figure
fig = go.Figure()

# Add left boundary to the figure
fig.add_trace(go.Scatter(
    x=gdf_left.geometry.x,
    y=gdf_left.geometry.y,
    mode='lines',
    name='Left Boundary',
    line=dict(color='blue')
))

# Add right boundary to the figure
fig.add_trace(go.Scatter(
    x=gdf_right.geometry.x,
    y=gdf_right.geometry.y,
    mode='lines',
    name='Right Boundary',
    line=dict(color='red')
))

# Add connections (nearest lines) to the figure
for line in nearest_lines_gdf.geometry:
    fig.add_trace(go.Scatter(
        x=[line.coords[0][0], line.coords[1][0]],
        y=[line.coords[0][1], line.coords[1][1]],
        mode='lines',
        name='Connection',
        line=dict(color='green', width=0.8)
    ))

# Set the plot layout
fig.update_layout(
    title='Track Boundaries and Nearest Connections',
    xaxis_title='X Coordinate',
    yaxis_title='Z Coordinate',
    showlegend=True,
    legend_title_text='Track Components',
    height=800,
)


# # Mid
default_zoom_x = [6.9365, 6.942]  # Replace these with your desired x-axis range
default_zoom_y = [50.331, 50.334]  # Replace 

fig.update_xaxes(range=default_zoom_x)
fig.update_yaxes(range=default_zoom_y)

# Show the figure
fig.show()


In [40]:

def summarize_graph(G):
    # Basic information
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    num_components = nx.number_connected_components(G)
    avg_degree = sum(dict(G.degree()).values()) / num_nodes
    
    # More detailed analysis
    components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
    largest_component = max(components, key=len)
    diameter = nx.diameter(largest_component) if num_components == 1 else 'Undefined for disconnected graphs'
    density = nx.density(G)
    
    # Summary
    print(f'Number of Nodes: {num_nodes}')
    print(f'Number of Edges: {num_edges}')
    print(f'Number of Connected Components: {num_components}')
    print(f'Average Degree: {avg_degree:.2f}')
    print(f'Diameter of the Largest Component: {diameter}')
    print(f'Graph Density: {density:.4f}')


def create_left_right_pairs(df_left, df_right):
    pass

def is_point_within_track():
    pass


def create_track_network(df_left_reduced, df_right_reduced, grid_distance, diagonally_connected):
    
    # Interpolate boundaries
    left_spline_x, left_spline_y, right_spline_x, right_spline_y = create_cubic_splines(df_left_reduced, df_right_reduced)
    
    # Determine the vertical bounds of the track
    min_y = min(df_left_reduced['Y-Coords'].min(), df_right_reduced['Y-Coords'].min())
    max_y = max(df_left_reduced['Y-Coords'].max(), df_right_reduced['Y-Coords'].max())
    
    # Create the grid of nodes, checking if each is within track boundaries
    valid_nodes = []
    for y in np.arange(min_y, max_y, grid_distance):
        for x in np.arange(df_left_reduced['X-Coords'].min(), df_right_reduced['X-Coords'].max(), grid_distance):
            if REPLACE WITH FUNCTION TO CHECK IF POINT IS WITHIN TRACK:
                valid_nodes.append((x, y))

    # Create the network graph from the valid nodes
    G = nx.Graph()
    for node in valid_nodes:
        G.add_node(node)
    
    # Add edges based on grid_distance, include diagonals if specified
    directions = [(0, grid_distance), (grid_distance, 0)]
    if diagonally_connected:
        directions.extend([(grid_distance, grid_distance), (-grid_distance, grid_distance)])
    
    for node in valid_nodes:
        x, y = node
        for dx, dy in directions:
            neighbor = (x + dx, y + dy)
            if neighbor in valid_nodes:
                G.add_edge(node, neighbor)
    
    return G


G = create_track_network(track_left_side, track_right_side, grid_distance = 5, diagonally_connected= True)
summarize_graph(G)

SyntaxError: invalid syntax (398175548.py, line 43)

In [ ]:

def visualize_and_save_graph(G, file_name, dpi=300):
    # Set the size of the figure
    ax.set_aspect('equal')
    fig.set_size_inches(15, 15)
    plt.show()
    
    # Extract node positions from the graph G
    pos = {node: (node[0], node[1]) for node in G.nodes()}
    
    # Draw the network
    nx.draw(G, pos, node_size=10, width=1, with_labels=False, node_color='blue', edge_color='gray')
    
    plt.show()
    # Save the figure
    # plt.savefig(file_name, format='png', dpi=dpi, bbox_inches='tight')
    # plt.close()  # Close the figure to free memory

# Example usage:
visualize_and_save_graph(G, 'Data/Map_details/track_network.png', dpi=300)
